### Import Necessary Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
df = pd.read_csv('dataset.csv')

In [4]:
df.sample(5)

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
749,יונדאי,2019,i10,1,אוטומטית,1250,בנזין,NaN,NaN,ירושלים והסביבה,ירושלים,68000.0,0.0,19/10/2022,19/10/2022,['יד ראשונה טסט מאי 2023 נהג יחיד'],NaN,44000,NaN,NaN
321,מאזדה,2013,מאזדה 3,3,אוטומטית,2000,בנזין,חברה,פרטית,קיסריה והסביבה,אור עקיבא,24000.0,2.0,12/07/2023,12/07/2023,['מזדה 3 מנוע 2000 הדגם עם 150 כוח סוס\nמתחייב...,שחור,200000,NaN,736.0
1475,סקודה,2018,אוקטביה,3,אוטומטית,1000,בנזין,חברה,פרטית,חיפה וחוף הכרמל,חיפה,75000.0,4.0,45071,45071,['רכב שמור ללא מעשנים בסלון נקיא ויפה'],לבן,181000,NaN,NaN
156,Lexsus,2010,לקסוס IS250,3,אוטומטית,2500,בנזין,NaN,NaN,אשדוד - אשקלון,אשדוד,65000.0,6.0,18/08/2022,18/08/2022,['רכב שנקנה מלקסוס ישראל-אצלי שנתיים-להניע ולנ...,NaN,125000,NaN,NaN
733,שברולט,2018,ספארק,1,אוטומטית,"1,400",בנזין,פרטית,פרטית,פתח תקוה והסביבה,פתח תקווה,59000.0,3.0,26/05/2022,02/06/2022,['למכירהשברולט ספארק 2018 הדגם המפואר LTZ רכב ...,שחור,66000,NaN,NaN


In [5]:
df.dtypes

manufactor          object
Year                 int64
model               object
Hand                 int64
Gear                object
capacity_Engine     object
Engine_type         object
Prev_ownership      object
Curr_ownership      object
Area                object
City                object
Price              float64
Pic_num            float64
Cre_date            object
Repub_date          object
Description         object
Color               object
Km                  object
Test                object
Supply_score       float64
dtype: object

### Convert and remove duplicates from the Data 

In [6]:
# Remove commas from 'capacity_Engine' and 'Km' if they are strings
if df['capacity_Engine'].dtype == 'object':
    df['capacity_Engine'] = df['capacity_Engine'].str.replace(',', '')

if df['Km'].dtype == 'object':
    df['Km'] = df['Km'].str.replace(',', '')

# Convert to appropriate data types
df['Year'] = df['Year'].astype('Int64')
df['Hand'] = df['Hand'].astype('Int64')
df['Price'] = df['Price'].astype('float64')
df['Pic_num'] = df['Pic_num'].astype('Int64')
df['Supply_score'] = df['Supply_score'].astype('Int64')
df['capacity_Engine'] = pd.to_numeric(df['capacity_Engine'], errors='coerce').astype('Int64')
df['Km'] = pd.to_numeric(df['Km'], errors='coerce').astype('Int64')



# Convert columns to category
df['Gear'] = df['Gear'].astype('category')
df['Engine_type'] = df['Engine_type'].astype('category')
df['Prev_ownership'] = df['Prev_ownership'].astype('category')
df['Curr_ownership'] = df['Curr_ownership'].astype('category')
df['Color'] = df['Color'].astype('category')

# Convert date columns to datetime64
df['Cre_date'] = pd.to_datetime(df['Cre_date'], errors='coerce')
df['Repub_date'] = pd.to_datetime(df['Repub_date'], errors='coerce')

# Ensure object types for remaining columns
df['manufactor'] = df['manufactor'].astype('object')
df['model'] = df['model'].astype('object')
df['Area'] = df['Area'].astype('object')
df['City'] = df['City'].astype('object')
df['Description'] = df['Description'].astype('object')
df['Test'] = df['Test'].astype('object')

# Check for duplicates and remove them
df = df.drop_duplicates(keep='first')

# Check the data types
df.dtypes


manufactor                 object
Year                        Int64
model                      object
Hand                        Int64
Gear                     category
capacity_Engine             Int64
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
Price                     float64
Pic_num                     Int64
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                object
Color                    category
Km                          Int64
Test                       object
Supply_score                Int64
dtype: object

### In this part, we will present how we chose to handle outliers and missing values in the numerical columns

##### Hand Column


In [7]:
df['Hand'].isnull().sum()

0

In [11]:
# We will check if there are extreme values to know whether to exclude them from the data
df['Hand'].value_counts()
# There are no extreme values

Hand
2     511
1     364
3     321
4     101
5      41
6      18
7       7
8       6
9       2
10      1
Name: count, dtype: Int64

In [13]:
# We grouped the data by 'Year' and aggregated the AVG 'Hand' for each group
year = df.groupby(['Year']).agg(hand_mean=('Hand', 'mean'),).reset_index()
year['hand_mean'] = year['hand_mean'].round().astype('Int64')
year

,Year,hand_mean
0,1983,3
1,1988,4
2,1990,5
3,1995,5
4,1998,3
5,1999,4
6,2000,3
7,2002,3
8,2003,3
9,2004,2


##### km Column


In [14]:
df['Km'].isnull().sum()

75

In [ ]:
# We will check if there are extreme values to know whether to exclude them from the data

# We assume that an extreme value is below 1,000,000 and above 700,000
(df['Km'] < 1000).sum() # 72 extreme values
# Multiply any value below 1000 by 100
(df['Km'] >500000).sum()# 4 - We will discard the 4 that are greater than 500000

In [ ]:
# Here we remove the extreme values in this column.
km_cleaned = df[df['Km'] <= 500000]
km_cleaned['Km'] = km_cleaned['Km'].apply(lambda x: x * 1000 if x < 1000 else x)

In [18]:
# We grouped the data by 'Year' and aggregated the AVG 'Km' for each group
km = km_cleaned.groupby(['Year']).agg(km_mean=('Km', 'mean'),).reset_index()
km['km_mean'] = km['km_mean'].round().astype('Int64')
km.head()

,Year,km_mean
0,1983,100000
1,1988,200000
2,1990,305000
3,1995,140000
4,1998,250000


##### capacity_engine Column


In [19]:
df['capacity_Engine'].isnull().sum()

26

In [ ]:
# We will check if there are extreme values to know whether to exclude them from the data

# We assume that engines with a volume that smaller than 800 is an extreme value, as well as over 8500 
(df['capacity_Engine'] < 800).sum() # 8 extreme value
(df['capacity_Engine'] > 6000).sum() # 6 extreme value


In [ ]:
# Here we remove the extreme values in this column.
engcap_cleaned = df[(df['capacity_Engine'] >= 800) & (df['capacity_Engine'] <= 6000)]

# Before starting, we will convert back the 2 categorical variables into objects in order to succeed in doing the group by.
# The problematic fields are Gear & Engine_type.
engcap_cleaned['Gear']= engcap_cleaned['Gear'].astype(str)
engcap_cleaned['Engine_type']= engcap_cleaned['Engine_type'].astype(str)

# Calculating the median for the clean data (without the extreme values) according to Group By 
engcap = engcap_cleaned.groupby(['manufactor', 'model', 'Year','Gear','Engine_type'])['capacity_Engine'].median().reset_index()
engcap = engcap.rename(columns={'capacity_Engine': 'median_capacity'})
engcap['median_capacity'] = engcap['median_capacity'].astype('Int64')



In [22]:
engcap.head()

,manufactor,model,Year,Gear,Engine_type,median_capacity
0,Lexsus,לקסוס CT200H,2012,אוטומטית,היברידי,1800
1,Lexsus,לקסוס GS300,2008,אוטומטית,בנזין,3000
2,Lexsus,לקסוס IS250,2008,אוטומטית,בנזין,2500
3,Lexsus,לקסוס IS250,2010,אוטומטית,בנזין,2500
4,Lexsus,לקסוס IS250,2011,אוטומטית,בנזין,2500


##### pic_num


In [23]:
df['Pic_num'].isnull().sum()
# We chose to fill the missing values with 0.

24

###### Test

In [25]:
df['Test'].isnull().sum()
# Due to a high percentage of missing values, we will remove this column from our data later on 

1274

##### Supply_score


In [26]:
df['Supply_score'].isnull().sum()
# Due to a high percentage of missing values, we will remove this column from our data later on 

1037

### Prepare the Data


In [ ]:
def prepare_data(df):
    # Add 'Unknown' to the categories fields before filling missing values
    for col in ['Prev_ownership', 'Curr_ownership', 'Color']:
        if 'Unknown' not in df[col].cat.categories:
            df[col] = df[col].cat.add_categories('Unknown')

    # Handle missing values for categorical columns
    df['Prev_ownership'] = df['Prev_ownership'].fillna('Unknown')
    df['Curr_ownership'] = df['Curr_ownership'].fillna('Unknown')
    df['Color'] = df['Color'].fillna('Unknown')
    df['Test'] = df['Test'].fillna('Unknown')

    # Merge auxiliary dataframes for missing value imputation
    df = df.merge(year, on='Year', how='inner')
    df['Hand'] = df['Hand'].fillna(df['hand_mean'])
    df = df.drop(columns=['hand_mean'])

    df = df.merge(engcap, on=['manufactor', 'model', 'Year', 'Gear', 'Engine_type'], how='inner')
    df['capacity_Engine'] = df['capacity_Engine'].fillna(df['median_capacity'])
    df = df.drop(columns=['median_capacity'])

    df = df.merge(km, on='Year', how='inner')
    df['Km'] = df['Km'].fillna(df['km_mean'])
    df = df.drop(columns=['km_mean'])

    df['Pic_num'] = df['Pic_num'].fillna(0)

    # Drop unnecessary columns before one-hot encoding
    df = df.drop(['Description', 'Cre_date', 'Repub_date', 'Supply_score', 'Test'], axis=1)
    
    # One-hot encode categorical variables
    categorical_cols = ['manufactor','model','Area','City','Color','Gear','Engine_type', 'Prev_ownership', 'Curr_ownership']
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

    # Ensure all columns are of numeric type where applicable
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], downcast='integer', errors='coerce').astype('Int64')

    return df
    
df_prepared = prepare_data(df)

In [30]:
df_prepared.head()

,Year,Hand,capacity_Engine,Price,Pic_num,Km,manufactor_אאודי,manufactor_אופל,manufactor_אלפא רומיאו,manufactor_ב.מ.וו,...,Prev_ownership_מונית,Prev_ownership_ממשלתי,Prev_ownership_פרטית,Prev_ownership_Unknown,Curr_ownership_השכרה,Curr_ownership_חברה,Curr_ownership_לא מוגדר,Curr_ownership_ליסינג,Curr_ownership_פרטית,Curr_ownership_Unknown
0,2015,2,1600,51000,2,144000,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,2018,1,1200,49000,0,69000,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,2010,1,1450,22500,1,145000,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,2016,1,1600,63000,5,27300,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,2012,1,1248,37000,1,70000,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


### Split Data into Features and Target

In [32]:
X = df_prepared.drop('Price', axis=1)
y = df_prepared['Price']


### Standardize Features

In [33]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


### Split into Train and Test Sets


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


###  Hyperparameter Tuning with Grid Search

In [35]:
# Initialize ElasticNet model
elastic_net = ElasticNet()

# Perform grid search for hyperparameter tuning
param_grid = {
    'alpha': [0.1, 1.0, 10.0],  
    'l1_ratio': [0.1, 0.5, 0.9]
}
grid_search = GridSearchCV(elastic_net, param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train )

# Get the best model from grid search
best_elastic_net = grid_search.best_estimator_

### Evaluate the Model with 10-Fold Cross-Validation

In [36]:
# Perform 10-fold cross-validation on the best model
cv_scores = cross_val_score(best_elastic_net, X_train, y_train, cv=10, scoring='neg_mean_squared_error')

### Train the Model on the Full Training Set

In [37]:
# Train the best model on the full training set
best_elastic_net.fit(X_train, y_train)


ElasticNet(l1_ratio=0.9)

### Evaluate the Model on the Test Set

In [38]:
# Evaluate the best model on the test set
y_pred = best_elastic_net.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Test RMSE:", test_rmse)


Test RMSE: 13224.486249853615


### Identify the Top 5 Features

In [41]:
# Identify the top 5 features
feature_importance = pd.Series(best_elastic_net.coef_, index=X.columns)
categorical_columns = ['manufactor','Year', 'model', 'Area', 'City', 'Color', 'Gear', 'Engine_type', 'Prev_ownership', 'Curr_ownership']
# Aggregate feature importances for the original categorical columns
aggregated_importance = {}

for col in feature_importance.index:
    if isinstance(col, str):
        base_col = col.split('_')[0]
        if base_col in categorical_columns:
            if base_col not in aggregated_importance:
                aggregated_importance[base_col] = abs(feature_importance[col])
            else:
                aggregated_importance[base_col] += abs(feature_importance[col])
        else:
            aggregated_importance[col] = abs(feature_importance[col])  # Ensure absolute value for consistency

# Convert to a pandas Series for easy sorting
aggregated_importance_series = pd.Series(aggregated_importance)

# Identify the top 5 original features
top_5_features_aggregated = aggregated_importance_series.abs().sort_values(ascending=False).head(5)
print("Top 5 Features Aggregated:\n", top_5_features_aggregated)

# Determine if the impact is positive or negative for each top feature
for feature in top_5_features_aggregated.index:
    impact = "positive" if aggregated_importance_series[feature] > 0 else "negative"
    print(f"Feature: {feature}, Impact: {impact}, Coefficient: {aggregated_importance_series[feature]}")


Top 5 Features Aggregated:
 model         134163.736413
City           59829.924534
manufactor     16785.646138
Color          12391.700667
Year           10956.808974
dtype: float64
Feature: model, Impact: positive, Coefficient: 134163.73641304564
Feature: City, Impact: positive, Coefficient: 59829.924533696016
Feature: manufactor, Impact: positive, Coefficient: 16785.64613790532
Feature: Color, Impact: positive, Coefficient: 12391.700667463143
Feature: Year, Impact: positive, Coefficient: 10956.808973730635
